In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
import os
import glob 

In [2]:
# 1. create folder 'clip-features-14' in each batch
# 2. uncomment :

# Phat 
filter_L = ['L01', 'L02', 'L03']

# Thang Nguyen 
filter_L = ['L04', 'L05', 'L06']

# Mai Dang Khoa
filter_L = ['L07', 'L08', 'L09']

# Thang Nguyen
filter_L = ['L10', 'L11', 'L12']

# 3. upload : https://drive.google.com/drive/folders/1bdVUUD8lsJv3_3aGBMBIzmfcqe9XaI3F?usp=share_link

In [3]:
# run in about 3 minutes in the first time installed
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")

/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# distinct videos
batch_videos_dict = {}
path_pattern = os.path.join(os.getcwd(), '..', '..', 'data', 
                            'batch*', 'keyframes', '*', '*')
paths = glob.glob(path_pattern)

for path in paths:
    _, batch_name, _, _, video_name = path.rsplit(os.sep, 4)
    if batch_name not in batch_videos_dict:
        batch_videos_dict[batch_name] = []
    batch_videos_dict[batch_name].append(video_name)

for batch in batch_videos_dict.keys():
    batch_videos_dict[batch].sort()

batch_videos_dict

{'batch1': ['L01_V001',
  'L01_V002',
  'L01_V003',
  'L01_V004',
  'L01_V005',
  'L01_V006',
  'L01_V007',
  'L01_V008',
  'L01_V009',
  'L01_V010',
  'L01_V011',
  'L01_V012',
  'L01_V013',
  'L01_V014',
  'L01_V015',
  'L01_V016',
  'L01_V017',
  'L01_V018',
  'L01_V019',
  'L01_V020',
  'L01_V021',
  'L01_V022',
  'L01_V023',
  'L01_V024',
  'L01_V025',
  'L01_V026',
  'L01_V027',
  'L01_V028',
  'L01_V029',
  'L01_V030',
  'L01_V031',
  'L02_V001',
  'L02_V002',
  'L02_V003',
  'L02_V004',
  'L02_V005',
  'L02_V006',
  'L02_V007',
  'L02_V008',
  'L02_V009',
  'L02_V010',
  'L02_V011',
  'L02_V012',
  'L02_V013',
  'L02_V014',
  'L02_V015',
  'L02_V016',
  'L02_V017',
  'L02_V018',
  'L02_V019',
  'L02_V020',
  'L02_V021',
  'L02_V022',
  'L02_V023',
  'L02_V024',
  'L02_V025',
  'L02_V026',
  'L02_V027',
  'L02_V028',
  'L02_V029',
  'L02_V030',
  'L02_V031',
  'L03_V001',
  'L03_V002',
  'L03_V003',
  'L03_V004',
  'L03_V005',
  'L03_V006',
  'L03_V007',
  'L03_V008',
  'L03_V00

In [5]:
# be filtered at here
for batch in batch_videos_dict.keys():
    batch_videos_dict[batch] = [video for video in batch_videos_dict[batch] if video[:3] in filter_L]

In [6]:
batch_videos_dict

{'batch1': ['L01_V001',
  'L01_V002',
  'L01_V003',
  'L01_V004',
  'L01_V005',
  'L01_V006',
  'L01_V007',
  'L01_V008',
  'L01_V009',
  'L01_V010',
  'L01_V011',
  'L01_V012',
  'L01_V013',
  'L01_V014',
  'L01_V015',
  'L01_V016',
  'L01_V017',
  'L01_V018',
  'L01_V019',
  'L01_V020',
  'L01_V021',
  'L01_V022',
  'L01_V023',
  'L01_V024',
  'L01_V025',
  'L01_V026',
  'L01_V027',
  'L01_V028',
  'L01_V029',
  'L01_V030',
  'L01_V031',
  'L02_V001',
  'L02_V002',
  'L02_V003',
  'L02_V004',
  'L02_V005',
  'L02_V006',
  'L02_V007',
  'L02_V008',
  'L02_V009',
  'L02_V010',
  'L02_V011',
  'L02_V012',
  'L02_V013',
  'L02_V014',
  'L02_V015',
  'L02_V016',
  'L02_V017',
  'L02_V018',
  'L02_V019',
  'L02_V020',
  'L02_V021',
  'L02_V022',
  'L02_V023',
  'L02_V024',
  'L02_V025',
  'L02_V026',
  'L02_V027',
  'L02_V028',
  'L02_V029',
  'L02_V030',
  'L02_V031']}

In [ ]:
for batch in batch_videos_dict.keys():
    for video in batch_videos_dict[batch]:
        path_save = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'clip-features-14', video + ".npy") 
        if os.path.isfile(path_save):
            continue

        path_to_keyframe = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'keyframes', 'keyframes_' + video[:3], video) 
        print(path_to_keyframe)

        embeddings = []
        image_files = [os.path.join(path_to_keyframe, f) for f in os.listdir(path_to_keyframe) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_files.sort()

        for image_path in image_files:
            print(image_path)
            image = Image.open(image_path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt")
            with torch.no_grad():
                image_features = model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            embeddings.append(image_features.cpu().numpy().flatten())
        embeddings_array = np.array(embeddings)
        path_save = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'clip-features-14', video + ".npy") 
        np.save(path_save, embeddings_array)